In [1]:
from dataset import TinyShakes

In [2]:
ts = TinyShakes()

tokens loaded from file ./data/encoded.bin
CDataset created...


In [3]:
X, y = ts[1]
print('X.shape: ', X.shape)
print(X)
print('y.shape: ', y.shape)
print(y)

X.shape:  torch.Size([3, 5])
tensor([[ 3421,   284, 44570,  8292,    11],
        [  557,   284, 15939,   416,   502],
        [  550,   517, 21213,   618,   314]])
y.shape:  torch.Size([3, 5])
tensor([[  284, 44570,  8292,    11,   198],
        [  284, 15939,   416,   502,    13],
        [  517, 21213,   618,   314,   938]])
